<a href="https://colab.research.google.com/github/mukhairatil/ABSA_BRIMO_GooglePlaystore_IndoRoBERTa/blob/main/absa_brimo_indoroberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>